In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('training.csv')
dd=pd.read_csv('test.csv')

In [3]:
df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
x=df['text']
y=df['airline_sentiment']
r=df['retweet_count']

In [5]:
from nltk.tokenize import word_tokenize 

In [7]:
documents=[]
for i in range(len(x)):
    documents.append((word_tokenize(x[i]),y[i]))
    for j in range(int(r[i])):
        documents.append((word_tokenize(x[i]),y[i]))

In [8]:
len(documents)

11869

In [9]:
documents[0]

(['@',
  'SouthwestAir',
  'I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes..not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 'negative')

In [10]:
from nltk.corpus import stopwords

In [11]:
stops=stopwords.words('english')

In [12]:
import string 
punctuation=list(string.punctuation)

In [13]:
stops=stops+punctuation

In [14]:
from nltk import pos_tag

In [15]:
li=[str(i) for i in range(10)]

In [16]:
stops=stops+li

In [17]:
from nltk.stem import WordNetLemmatizer

In [18]:
lemmatizer=WordNetLemmatizer()

In [19]:
from nltk.corpus import wordnet
def get_simple_word(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    else:
        return wordnet.NOUN

In [20]:
def clean_review(words):
    output_word=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_word(pos[0][1]))
            output_word.append(clean_word)
    return output_word

In [21]:
documents=[(clean_review(doc),category) for doc,category in documents]

In [22]:
Y_train=[category for doc,category in documents]

In [23]:
text_documents=[' '.join(doc) for doc,category in documents]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2))

In [26]:
X_train=count_vec.fit_transform(text_documents)

In [27]:
x_test=dd['text']

In [28]:
documents_test=[]
for i in range(len(x_test)):
    documents_test.append(word_tokenize(x_test[i]))

In [29]:
documents_test=[clean_review(doc) for doc in documents_test]

In [30]:
text_documents=[' '.join(doc) for doc in documents_test]

In [31]:
X_test=count_vec.transform(text_documents)

In [32]:
from sklearn.svm import SVC

In [33]:
svc=SVC(C=100,gamma=0.001)

In [34]:
svc.fit(X_train,Y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [35]:
svc.score(X_train,Y_train)

0.8746313927036818

In [36]:
y_pred=svc.predict(X_test)

In [37]:
s=pd.DataFrame(y_pred)

In [38]:
s.to_csv('predI.csv',header=False,index=False)